In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd 
import pymongo 
from jsonpath import jsonpath
import sys 
from typing import List
from pydantic import BaseModel, Field
# sys.path.append("..")
os.chdir("G:/hansagt/ecommerce/backend")

dotenv_path = 'conf/dev.env'
load_dotenv(dotenv_path=dotenv_path, override=True, verbose=True)

In [ ]:
print(os.getenv('DEBUG'))
from vo.carriers import PickSlipItemVO
from core.db import OrderQueryParams
from rest.pickpack.DataManager import PickPackMongoDBManager

In [ ]:
with PickPackMongoDBManager("192.168.8.10", 27017) as man:
    # amazon_unshipped = man.find_unshipped_amazon_orders()
    # orders = amazon_unshipped
    params = OrderQueryParams()
    params.status = ["Shipped"]
    orders = man.amazonDataManager.find_orders_by_query_params(params)
    refs = [order.orderId for order in orders]
    items = man.get_pick_items_by_references(refs=refs)
    packrefs = man.sort_packing_order_refs(refs=refs)

    orders = man.amazonDataManager.find_orders_by_ids(packrefs)

In [ ]:
dic_items = map(lambda x: x.dict(), items)
df = pd.DataFrame.from_dict(dic_items).sort_values(by='orderId')
df.head(10)

## Picking Slip

In [ ]:
df_slip = df.groupby(['sku', 'title'], as_index=False) \
                   .agg({'quantity': 'sum', 'sku': 'first', 'orderKey': 'first', 'storageLocation': 'first'})
df_slip.sort_values(by=['sku', 'title'], inplace=True)
# Reorder columns
df_slip = df_slip[['sku', 'quantity','storageLocation', 'orderKey', 'title']]
# Save to Excel
df_slip.to_excel('scripts/pick_slip.xlsx', index=False)
df_slip

In [ ]:
slips = df_slip.to_dict(orient='records')
slips = list(map(lambda x: PickSlipItemVO(**x), slips))

## Packing Slip 生成Excel

In [ ]:
# Order ids of sorted orders to be packed
# packrefs
import io
order_dict = {order.orderId: order for order in orders}
ordered_results = [order_dict[order_id] for order_id in packrefs]

df_order_lines = []

for i, o in enumerate(ordered_results):    
    for line in o.items:
        o_item = PickSlipItemVO(orderId=o.orderId, taskId=i+1, sku=line.sku, quantity=line.quantity, title=line.name)
        df_order_lines.append(o_item)

df_order_lines = [ o.dict() for o in df_order_lines]
df_order_lines = pd.DataFrame.from_dict(df_order_lines)
# Reduced columns
df_order_lines = df_order_lines[['taskId', 'orderId', 'sku', 'quantity', 'title']]

#df_order_lines
display(df_order_lines)
excel_file = "scripts/pack_orders.xlsx"
excel_bytes = io.BytesIO()
df_order_lines.to_excel(excel_bytes, index=False, sheet_name="pack_orders")
# df_order_lines.to_excel(excel_file, index=False, sheet_name="pack_orders")

In [ ]:
# 加载生成的Excel文件以便修改
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import PatternFill

wb = load_workbook(excel_bytes)
ws = wb.active

# 定义背景填充样式
gray_fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
white_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")

# 遍历所有行，并根据taskId设置背景色
for row in range(2, ws.max_row + 1):  # 从第二行开始，因为第一行是表头
    task_id = ws.cell(row=row, column=1).value
    fill = gray_fill if task_id % 2 == 0 else white_fill
    for col in range(1, ws.max_column + 1):
        ws.cell(row=row, column=col).fill = fill

# 保存修改后的Excel文件
wb.save(excel_file)